In [1]:
import numpy as np
import spikeinterface.full as si
import peak_detection

In [2]:
folder_path = "../data/128_channels"
sampling_frequency, num_channels, file_paths = peak_detection.retrieve_parameters(folder_path + "/recording")
recording = si.load(folder_path + "/recording")
sorting = si.load(folder_path + "/sorting")
templates = np.load(folder_path + "/templates.npy")

In [3]:
from spikeinterface.sortingcomponents.tools import get_prototype_and_waveforms
prototype, wfs, _ = get_prototype_and_waveforms(recording, ms_before=1, ms_after=3)
import numpy as np
n_components = 5

noise_level (no parallelization):   0%|          | 0/20 [00:00<?, ?it/s]

get protoype waveforms (no parallelization):   0%|          | 0/50 [00:00<?, ?it/s]

In [4]:
from sklearn.cluster import KMeans
wfs /= np.linalg.norm(wfs, axis=1)[:, None]
model = KMeans(n_clusters=n_components, n_init=10).fit(wfs)
temporal_components = model.cluster_centers_
temporal_components = temporal_components / np.linalg.norm(temporal_components, axis=1)[:, None]
temporal_components = temporal_components.astype(np.float32)

In [5]:
from sklearn.decomposition import TruncatedSVD
model = TruncatedSVD(n_components=n_components).fit(wfs)
spatial_components = model.components_.astype(np.float32)

In [6]:
data = recording.get_traces().astype(np.float32)

In [15]:
si_templates = si.Templates(templates, 
                            sampling_frequency=recording.get_sampling_frequency(), 
                            channel_ids=recording.get_channel_ids(), 
                            unit_ids=sorting.get_unit_ids(),
                            nbefore=int(sampling_frequency//1000),)

In [8]:
si_templates

Templates: 100 units - 100 samples - 128 channels 
sampling_frequency=25.00 kHz - ms_before=1.00 ms - ms_after=3.00 ms

In [9]:
recording

GroundTruthRecording (BinaryFolderRecording): 128 channels - 25.0kHz - 1 segments 
                      1,250,000 samples - 50.00s - float32 dtype - 610.35 MiB

In [16]:
from spikeinterface.sortingcomponents.matching import find_spikes_from_templates
spikes = find_spikes_from_templates(recording, si_templates, method='kilosort-matching', method_kwargs={'temporal_components': temporal_components, 
                                                                                                        'spatial_components': spatial_components})

find spikes (kilosort-matching) (no parallelization):   0%|          | 0/50 [00:00<?, ?it/s]

In [18]:
spikes.shape

(443179,)

In [24]:
spikes = find_spikes_from_templates(recording, si_templates, method='kilosort-matching', method_kwargs={'temporal_components': temporal_components, 
                                                                                                        'spatial_components': spatial_components},
                                                                                                        job_kwargs={'n_jobs':20})

find spikes (kilosort-matching) (workers: 20 processes):   0%|          | 0/50 [00:00<?, ?it/s]

In [21]:
spikes.shape

(443188,)

In [ ]:
spikes = find_spikes_from_templates(recording, si_templates, method='kilosort-matching', method_kwargs={'temporal_components': temporal_components, 
                                                                                                        'spatial_components': spatial_components,
                                                                                                        'torch_device':"cuda"},
                                                                                                        job_kwargs={'n_jobs':20})